In [1]:
#from tensorflow.keras.processing.image import load_img
import numpy as np
import matplotlib.pyplot as plt
 
%matplotlib inline
 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import SGD
print(tf.__version__)
#!pip install unzip

2.17.1


In [2]:
#!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
import zipfile
import os

# Path to your ZIP file
zip_file_path = "data.zip"

# Destination folder (change if needed)
destination_folder = "C:/Users/Kuzey"

# Ensure the destination folder exists
os.makedirs(destination_folder, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)

print(f"Files extracted to {destination_folder}")

Files extracted to C:/Users/Kuzey


In [3]:
#!pip install tensorflow==2.17.1

In [4]:
import numpy as np
import tensorflow as tf

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

Model
For this homework we will use Convolutional Neural Network (CNN). Like in the lectures, we'll use Keras.

You need to develop the model with following structure:

- The shape for input should be (200, 200, 3)
- Next, create a convolutional layer (Conv2D):
- Use 32 filters
- Kernel size should be (3, 3) (that's the size of the filter)
- Use 'relu' as activation
- Reduce the size of the feature map with max pooling (MaxPooling2D)
- Set the pooling size to (2, 2)
- Turn the multi-dimensional result into vectors using a Flatten layer
- Next, add a Dense layer with 64 neurons and 'relu' activation
- Finally, create the Dense layer with 1 neuron - this will be the output
- The output layer should have an activation - use the appropriate activation for the binary classification case
- As optimizer use SGD with the following parameters:

- SGD(lr=0.002, momentum=0.8)

In [5]:
from tensorflow import keras
from tensorflow.keras import layers

def make_model(learning_rate=0.002, size_inner=64):
    # Input layer
    inputs = keras.Input(shape=(200, 200, 3))
    
    # Convolutional layer with 32 filters and ReLU activation
    x = layers.Conv2D(
        filters=32, 
        kernel_size=(3, 3), 
        activation='relu'
    )(inputs)
    
    # MaxPooling layer to reduce feature map size
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    
    # Flatten layer to convert feature maps into a 1D vector
    x = layers.Flatten()(x)
    
    # Dense layer with 64 neurons and ReLU activation
    x = layers.Dense(size_inner, activation='relu')(x)
    
    # Output Dense layer with 1 neuron and sigmoid activation
    outputs = layers.Dense(1, activation='sigmoid')(x)  # Sigmoid for binary classification
    
    # Create the model
    model = keras.Model(inputs, outputs)
    
    # Compile the model with SGD optimizer
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.8)
    loss = keras.losses.BinaryCrossentropy()  # Sigmoid automatically matches this loss
    
    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy']
    )
    
    return model

# Create the model
cnn_model = make_model()

# Print the model summary
cnn_model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 200, 200, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 198, 198, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 99, 99, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 313632)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │      20,072,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,073,473 (76.57 MB)

 Trainable params: 20,073,473 (76.57 MB)

 Non-trainable params: 0 (0.00 B)

__Q1.__ Since we have a binary classification problem, what is the best loss function for us?

- mean squared error
- __binary crossentropy__
- categorical crossentropy
- cosine similarity

__Q2.__  What's the total number of parameters of the model? You can use the summary method for that.

- 896
- 11214912
- 15896912
- __20072512__

For the next two questions, use the following data generator for both train and test sets:

`ImageDataGenerator(rescale=1./255)`

We don't need to do any additional pre-processing for the images.

When reading the data from train/test directories, check the class_mode parameter. Which value should it be for a binary classification problem?
Use `batch_size=20`

Use shuffle=True for both training and test sets.

For training use `.fit()` with the following params:


`model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)`

In [6]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
 
train_ds = train_gen.flow_from_directory(
    './data/train',
    target_size=(200, 200),
    batch_size=20,
    shuffle=True,
    class_mode='binary'
   
)
 
val_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
 
val_ds = val_gen.flow_from_directory(
    './data/test',
    target_size=(200, 200),
    batch_size=20,
    shuffle=True,
    class_mode='binary'

)

Found 800 images belonging to 2 classes.
Found 201 images belonging to 2 classes.


In [7]:

 
 
history = cnn_model.fit(train_ds, epochs=10,  validation_data=val_ds)

C:\Users\Kuzey\Anaconda3\envs\ml-zoomcamp\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 19s 445ms/step - accuracy: 0.5635 - loss: 0.6878 - val_accuracy: 0.6119 - val_loss: 0.6316
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 16s 399ms/step - accuracy: 0.6427 - loss: 0.6004 - val_accuracy: 0.6617 - val_loss: 0.6180
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 16s 396ms/step - accuracy: 0.7458 - loss: 0.5292 - val_accuracy: 0.6368 - val_loss: 0.6317
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 16s 395ms/step - accuracy: 0.7157 - loss: 0.5610 - val_accuracy: 0.6617 - val_loss: 0.6382
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 16s 392ms/step - accuracy: 0.7628 - loss: 0.5000 - val_accuracy: 0.6418 - val_loss: 0.6328
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 16s 389ms/step - accuracy: 0.7535 - loss: 0.4938 - val_accuracy: 0.6617 - val_loss: 0.6390
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 16s 393ms/step - accuracy: 0.7706 - loss: 0.4888 - val_accuracy: 0.6716 - val_loss: 0.6228
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 16s 394ms/step - accuracy: 0.7845 - loss: 0.4621 - val_accu

In [8]:
import numpy as np


training_accuracy = history.history['accuracy']  
training_loss = history.history['loss'] 


median_accuracy = np.median(training_accuracy)


std_dev_loss = np.std(training_loss)

print(f"Median of Training Accuracy: {median_accuracy:.2f}")
print(f"Standard Deviation of Training Loss: {std_dev_loss:.3f}")

Median of Training Accuracy: 0.76
Standard Deviation of Training Loss: 0.079


__Question 3__
What is the median of training accuracy for all the epochs for this model?

- 0.10
- 0.32
- 0.50
- __0.72__


__Question 4__
What is the standard deviation of training loss for all the epochs for this model?

- 0.028
- __0.068__
- 0.128
- 0.168

__Data Augmentation__

For the next two questions, we'll generate more data using data augmentations.

Add the following augmentations to your training data generator:

- `rotation_range=50,`
- `width_shift_range=0.1,`
- `height_shift_range=0.1,`
- `zoom_range=0.1,`
- `horizontal_flip=True,`
- `fill_mode='nearest'`

In [9]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [10]:

 
train_ds_new = train_gen.flow_from_directory(
    './data/train',
    target_size=(200, 200),
    batch_size=20,
    shuffle=True,
    class_mode='binary'
   
)


Found 800 images belonging to 2 classes.


In [11]:
# Continue training the model for 10 more epochs
history = cnn_model.fit(
    train_ds_new,                     # Augmented data generator
    batch_size=20,
    epochs=10,                    # Train for 10 more epochs
    validation_data=val_ds# Validation data (assumed to be prepared)
)
   

# Optionally, you can save the updated model
# model.save('path_to_save_model')  # Save the updated model

# Calculate the mean of test loss for all the epochs
test_loss = history.history['val_loss']  # Get the validation loss history
mean_test_loss = np.mean(test_loss)     # Calculate the mean of the test loss

print(f'Mean test loss over 10 additional epochs: {mean_test_loss}')

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 21s 531ms/step - accuracy: 0.6959 - loss: 0.6192 - val_accuracy: 0.7114 - val_loss: 0.5503
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 22s 542ms/step - accuracy: 0.6548 - loss: 0.6101 - val_accuracy: 0.7413 - val_loss: 0.5485
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 22s 541ms/step - accuracy: 0.7104 - loss: 0.5635 - val_accuracy: 0.7164 - val_loss: 0.5755
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 21s 517ms/step - accuracy: 0.7049 - loss: 0.5697 - val_accuracy: 0.6567 - val_loss: 0.6407
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 21s 524ms/step - accuracy: 0.6846 - loss: 0.5724 - val_accuracy: 0.7114 - val_loss: 0.5692
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 21s 520ms/step - accuracy: 0.7488 - loss: 0.5490 - val_accuracy: 0.7264 - val_loss: 0.5331
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 21s 522ms/step - accuracy: 0.7095 - loss: 0.5778 - val_accuracy: 0.6766 - val_loss: 0.6209
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 21s 526ms/step - accuracy: 0.7314 - loss: 0.5377 - val_accu

Let's train our model for 10 more epochs using the same code as previously.

Note: make sure you don't re-create the model - we want to continue training the model we already started training.

__Question 5__ What is the mean of test loss for all the epochs for the model trained with augmentations?

- 0.26
- __0.56__
- 0.86
- 1.16

In [12]:
import numpy as np

# Assuming 'history' is the object returned from model.fit() after training for 10 more epochs
val_accuracy = history.history['val_accuracy']  # Get the validation accuracy history

# Extract the validation accuracy for epochs 6 to 10 (index 5 to 9)
val_accuracy_last_5_epochs = val_accuracy[5:10]  # Epochs 6 to 10 correspond to indices 5 to 9

# Calculate the average of test accuracy for epochs 6 to 10
average_val_accuracy = np.mean(val_accuracy_last_5_epochs)

print(f'Average test accuracy for epochs 6 to 10: {average_val_accuracy:.4f}')

Average test accuracy for epochs 6 to 10: 0.7204


In [13]:
print(history.history['val_accuracy'] )

[0.711442768573761, 0.7412935495376587, 0.7164179086685181, 0.6567164063453674, 0.711442768573761, 0.7263681888580322, 0.676616907119751, 0.7213930487632751, 0.7363184094429016, 0.7412935495376587]


__Question 6__ What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?

- 0.31
- 0.51
- __0.71__
- 0.91